<a href="https://colab.research.google.com/github/preeti13456/machine-learning-/blob/master/Ion_Switchingipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.optimizers import SGD, RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam
from keras.preprocessing.image import ImageDataGenerator


In [0]:
df = pd.read_csv('sample_submission.csv')
df.head()

,time,open_channels
0,500.0001,0.0
1,500.0002,0.0
2,500.0003,0.0
3,500.0004,0.0
4,500.0005,0.0


In [0]:
max_quantity = df['open_channels'].max()
min_quantity = df['open_channels'].min()

In [0]:
df['open_channels'] = (df['open_channels']-min_quantity)/(max_quantity-min_quantity)
df_quan = pd.DataFrame(columns= ['time'])
df_quan.time = df.time
df.head()

,time,open_channels
0,500.0001,NaN
1,500.0002,NaN
2,500.0003,NaN
3,500.0004,NaN
4,500.0005,NaN


In [0]:
x  = pd.read_csv('train.csv')
x.head()

,time,signal,open_channels
0,0.0001,-2.7600,0.0
1,0.0002,-2.8557,0.0
2,0.0003,-2.4074,0.0
3,0.0004,-3.1404,0.0
4,0.0005,-3.1525,0.0


In [0]:
y = pd.read_csv('test.csv')
y.head()

,time,signal
0,500.0001,-2.6498
1,500.0002,-2.8494
2,500.0003,-2.8600
3,500.0004,-2.4350
4,500.0005,-2.6155


In [0]:
df_x = df.to_numpy()
df_y = df_quan.to_numpy()
# separate into train validation and testing data.
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, shuffle= False)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle= False)
x_train = x_train[:-1]
x_val = x_val[:-1]
x_test = x_test[:-1]
y_train = y_train[1:]
y_val = y_val[1:]
y_test = y_test[1:]
print(x_train.shape, x_val.shape, x_test.shape,
      y_train.shape, y_val.shape, y_test.shape)

(114388, 2) (38129, 2) (38129, 2) (114388, 1) (38129, 1) (38129, 1)


In [0]:
train_x = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
test_x = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
val_x = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(train_x.shape, val_x.shape, test_x.shape,
      y_train.shape, y_val.shape, y_test.shape)

In [0]:
#x_train, x_val, y_train, y_val = train_test_split(df_x, df_y, test_size = 0.25, shuffle=False)
df_x = df_x[:-1]
df_y = df_y[:-1]

NameError: ignored

In [0]:
df_x.shape

In [0]:
df_y.shape

NameError: ignored

In [0]:
train_x = df_x.reshape((df_x.shape[0], 1, df_x.shape[1]))

In [0]:
test_y = df_y.reshape((df_y.shape[0], 1, df_y.shape[1]))

In [0]:
print(train_x.shape,test_y.shape)

In [0]:
#Designing The lstm model
epoch = 1500
batch = 32
decay = 200
learning_rate = 0.003
neurons = 17
dropout = 0
output_neurons = 1
version = 0

In [0]:
file_path = "\preeti sharma\Documents\liverpool-ion-switching"  + "_V_" + str(version) + "_L1_"  + str(neurons)  + "_Dro1_" + str(dropout) + "De1" + str(output_neurons)  +"a.hdf5"

In [0]:
model = Sequential()
model.add(LSTM(neurons, input_shape=(1,train_x.shape[2]), return_sequences=False, name='L1'))

In [0]:
model.add(Dense(output_neurons, name='De1'))

In [0]:
model.add(Activation('relu'))

In [0]:
#adam = optimizers.Adam(lr = learning_rate, decay=decay)
#model.compile(loss = 'mean_squared_error', optimizer=adam)
opt = tf.keras.optimizers.Adam(lr=learning_rate, decay=decay)

model.compile(optimizer=opt, loss='mean_squared_error', metrics=["accuracy"])

In [0]:
checkpoint = ModelCheckpoint(file_path, monitor='val_loss',
                             verbose=1, 
                             save_best_only = True,
                             mode='auto', period = 1)

In [0]:
datagen = ImageDataGenerator(
    rotation_range=3,
#     featurewise_std_normalization=True,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
#history = model.fit(train_x, y_train, batch_size=batch, epochs=epoch, verbose=3, validation_data=(val_x,y_val), callbacks=[checkpoint])
history = model.fit_generator( datagen.flow( train_x, y_train, batch_size = batch, shuffle = True),
        samples_per_epoch = epoch, callbacks = [checkpoint],
        validation_data = val_datagen.flow( val_x, y_val ),  nb_val_samples=val_x.shape[0], verbose = 1, show_accuracy = True )